In [1]:
from google.auth.transport import requests
from googleapiclient import discovery
from google.oauth2 import service_account
from google.cloud import storage
from googleapiclient.http import MediaIoBaseDownload
import io
import threading
from time import sleep
import pandas as pd

In [2]:
from datetime import time

drivers_check_list = []
drivers_checked_list = []
ls_threads = []
ls_tmp = []
ls_error = []
ls_error_files = []
ls_files_to_download = []
count_files_checked = 0
count_files_down = 0
count_files_error = 0

drivers = [['Ci', '0AHj-PRcbOxSWUk9PVA'],
             ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
             # ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
             ['Архив', '0AOP_oKFy4oGJUk9PVA'],
             ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
             ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
             # ['Контент', '0ANf3LpamTrsTUk9PVA'],
             # ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
             ['Объединенная пивоварня', '0AELFNnmNadJqUk9PVA'],
             ['ОСК Шереметьево', '0ALRUo2DlF1YmUk9PVA'],
             ['Мой Диск _', '0AHpbUawYyiEZUk9PVA']
           ]

def transfer_drive(drive_name_id):
    global count_files_down, count_files_error, count_files_checked
    while True:
        is_download_files = True

        try:
            service, bucket = get_client()
            ls_files = get_list_of_files_in_drive(service, drive_name_id)
            for file in ls_files:
                count_files_checked += 1
                done = download_file(service, bucket, drive_name_id, file)
                if done == 1:
                    is_download_files = False
                # try:
                #     done = download_file(service, bucket, drive_name_id, file)
                #     if done == 1:
                #         is_download_files = False
                # except Exception as e:
                #     ls_error.append([drive_name_id, get_full_path_to_file(service, drive_name_id,file), e])

            ls_tmp.append(f'{drive_name_id[0]}  - Завершено  - {is_download_files}')

            if is_download_files:
                drivers_checked_list.append(drive_name_id[1])
                break

        except Exception as e:
            ls_error.append([drive_name_id, 'Ошибка', e])





def transfer_drive_thread():
    while True:
        try:
            service, bucket = get_client()
            ls_drives = get_list_of_drives(service)
            s_drives = 'TT '
            drivers1 = []
            for l in ls_drives:
                for d in drivers:
                    if l[0] == d[0]:
                        drivers1.append(l)

            for drive in drivers1:
                if drive[1] not in drivers_check_list and drive[1] not in drivers_checked_list:
                    drivers_check_list.append(drive[1])
                    t = threading.Thread(target=transfer_drive, args=(drive,))
                    t.start()
                    ls_threads.append(t)
                    s_drives += drive[0] + ', '
        except Exception as e:
            ls_error.append(['get_list_of_drives', e])
        ls_tmp.append(f'Завершено------------------------{s_drives}')

        try:
            sleep(60*10)
        except Exception as e:
            ls_error.append(e)



def transfer_drives_thread():
    t = threading.Thread(target=transfer_drive_thread)
    ls_threads.append(t)
    t.start()

    return t


def get_client():
    # Google Drive
    credentials_drive = service_account.Credentials.from_service_account_file(
        filename='savvy-concord-374520-bbbb0e3c73c3.json',
        scopes=['https://www.googleapis.com/auth/drive'],
    )
    session = requests.AuthorizedSession(credentials_drive)
    service = discovery.build('drive', 'v3', credentials=credentials_drive)

    #Google Storage
    credentials_storage = service_account.Credentials.from_service_account_file(
        filename='credentials.json',
        scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
    )
    client = storage.Client(credentials=credentials_storage)
    bucket = client.get_bucket('cabinet_bucket')

    return service, bucket

# Общие диски
def get_list_of_drives(service):
    response = service.drives().list(pageSize=100).execute()
    ids = []
    for drive in response['drives']:
        ids.append([drive['name'], drive['id']])
    return ids

def get_list_of_files_in_drive(service, drive_name_id):
    ls_files = []

    response = service.files().list(
        q="mimeType!='application/vnd.google-apps.folder'",
        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='nextPageToken, files(id, name, mimeType)').execute()


    while response:
        for file in response.get('files', []):
            # s_path =get_full_path_to_file(drive_name_id, file.get('id'))
            # ls_files.append(file.get('id'))
            if file['mimeType'] != 'application/vnd.google-apps.document' and file['mimeType'] != 'application/vnd.google-apps.spreadsheet':
                ls_files.append(file.get('id'))


        page_token = response.get('nextPageToken', None)

        if page_token is None:
            break

        response = service.files().list(
            q="mimeType!='application/vnd.google-apps.folder'",
            corpora='drive',
            driveId=drive_name_id[1],
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token).execute()
    return ls_files


# Полный путь к файлу
def get_full_path_to_file(service, drive_name_id, file_id):
    s_main_folder = 'GoogleDriveTransfer/'
    ls = get_full_path_to_file_list(service, drive_name_id[1], file_id, [])
    ls.reverse()
    ls[0] = drive_name_id[0]
    s_path = s_main_folder + '/'.join(ls)
    return s_path


def get_full_path_to_file_list(service, drive_id, file_id, ls):
    response = service.files().get(
        fileId=file_id,
        fields='id, name, parents',
        supportsAllDrives=True).execute()
    file = response
    # print(file['name'])
    ls.append(file['name'])
    if file.get('parents') is not None:
        for parent in file['parents']:
            get_full_path_to_file_list(service, drive_id, parent, ls)
    return ls

# Скачать файл
def download_file(service, bucket, drive_name_id, file_id):
    global count_files_error, count_files_down
    file_path = get_full_path_to_file(service, drive_name_id, file_id)

    try:
        blobs = bucket.list_blobs(prefix=file_path)
        files = [blob.name for blob in blobs]
        if file_path in files:
            # print('Файл уже существует')
            return 0

         # request = service.files().get_media(fileId=file_id).execute()
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("Download %d%%." % int(status.progress() * 100))

        blob = bucket.blob(file_path)
        blob.upload_from_string(fh.getvalue())
        # blob.upload_from_string(request)
        ls_files_to_download.append(file_path)
        count_files_down += 1
    except Exception as e:
        ls_error_files.append([drive_name_id, file_path, file_id, 'Ошибка при скачивании', e])
        count_files_error += 1
        return 0

    return 1

In [3]:
t = transfer_drives_thread()

In [4]:
service, bucket = get_client()

In [17]:
def get_list_of_files_in_drive1(service, drive_name_id):
    ls_files = []

    response = service.files().list(
        q="mimeType='application/vnd.google-apps.document'",
        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='nextPageToken, files(id, name, mimeType)').execute()


    while response:
        for file in response.get('files', []):
            ls_files.append([file.get('id'), file.get('name'), file.get('mimeType')])


        page_token = response.get('nextPageToken', None)

        if page_token is None:
            break

        response = service.files().list(
            q="mimeType='application/vnd.google-apps.document'",
            corpora='drive',
            driveId=drive_name_id[1],
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token).execute()
    return ls_files

In [5]:
ls_drives = get_list_of_drives(service)

In [10]:
drive_name_id = ls_drives[-1]

Download 100%.


In [13]:
response = service.files().list(

        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='files(id, name, mimeType, parents)').execute()

Download 100%.


In [44]:
drivers = [['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объекты в работе', '0AOxaPQRooOk2Uk9PVA'],
 ['Остров мечты ППР', '0AImPD4jQdMXJUk9PVA'],
 ['Пекарня', '0AIWIhafOWgmHUk9PVA'],
 ['ПОДРЯДЧИКИ', '0ALnZgMLQFfrtUk9PVA'],
 ['СТО ГБО', '0AF2NtBbx_XvYUk9PVA'],
 ['ТЕХВАГОНМАШ', '0AAAyvFQMbstZUk9PVA'],
 ['Транспроект', '0ADKHIbPu575TUk9PVA'],
 ['ФосАгро', '0AD-4c3pLaBV5Uk9PVA'],
 ['ФРАКДЖЕТ-ТУЛЗ', '0ACjAWTbTqvESUk9PVA'],
 ['Холодный склад', '0ALT2Y1wkG7vCUk9PVA'],
 ['Чайка Лаб', '0APbGeMNQtN3MUk9PVA'],
 ['Читинские ключи', '0AA9gFHffXEitUk9PVA'],
 ['Юрист UA', '0AGg7JoJwa_8nUk9PVA']]

In [45]:
for dr in ls_drives:
    if dr not in drivers:
        print(dr)

['Ci', '0AHj-PRcbOxSWUk9PVA']
['Managment 2', '0AE3Z9Ox2YaurUk9PVA']
['VG.COM', '0AElRg0lTw8iIUk9PVA']
['Архив', '0AOP_oKFy4oGJUk9PVA']
['Винный парк', '0ANhu4lfOrlBiUk9PVA']
['Контент_VG', '0AN6chrg4ApnnUk9PVA']
['Контент', '0ANf3LpamTrsTUk9PVA']
['Лобня', '0AIQWy4IyyeaIUk9PVA']
['Мой Диск _', '0AHpbUawYyiEZUk9PVA']
['Объединенная пивоварня', '0AELFNnmNadJqUk9PVA']
['ОСК Шереметьево', '0ALRUo2DlF1YmUk9PVA']


In [6]:
drive_name_id = ls_drives[0]

In [18]:
ls_files = get_list_of_files_in_drive1(service, drive_name_id)

In [21]:
fl1 = ls_files[0]
fl1_id = fl1[0]
fl1_mime = fl1[2]
fl1

['1SKwTfRGC4Td_4K5ClSIlOLRsiE59vLIwSegzgovQOGw',
 'Учет рабочего времени. Методика работы по Тайм трекингу CRM системы',
 'application/vnd.google-apps.document']

In [43]:
request = service.files().export(fileId=fl1_id, mimeType=fl1_mime)

In [ ]:
resp = request.execute(http=http)

In [25]:
mime_types = [
    'application/vnd.google-apps.audio',
    'application/vnd.google-apps.document',
    'application/vnd.google-apps.drive-sdk',
    'application/vnd.google-apps.drawing',
    'application/vnd.google-apps.file',
    'application/vnd.google-apps.form',
    'application/vnd.google-apps.fusiontable',
    'application/vnd.google-apps.jam',
    'application/vnd.google-apps.map',
    'application/vnd.google-apps.photo',
    'application/vnd.google-apps.presentation',
    'application/vnd.google-apps.script',
    'application/vnd.google-apps.shortcut',
    'application/vnd.google-apps.site',
    'application/vnd.google-apps.spreadsheet',
    'application/vnd.google-apps.unknown',
    'application/vnd.google-apps.video'
]

In [7]:
response = service.files().list(
        q=f"mimeType='application/vnd.google-apps.document' or mimeType='application/vnd.google-apps.spreadsheet'",
        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='files(id, name, mimeType)').execute()

Download 100%.


In [8]:
files = response.get('files', [])

In [9]:
ls4 = []
ls4 += files
ls4 += files


In [37]:
ls1 = {}
for drive_name_id in ls_drives:
    ls2 = []
    for mime_type in mime_types:
        response = service.files().list(
            q=f"mimeType='{mime_type}'",
            corpora='drive',
            driveId=drive_name_id[1],
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            fields='files(id, name, mimeType)').execute()
        ls2.append([mime_type, len(response.get('files', []))])
    ls1[drive_name_id[0]] = ls2

In [42]:
ls3 = {}
for key, val in ls1.items():
    for i in val:
       if i[1] != 0:
           print(key, i)

BackOffice ['application/vnd.google-apps.document', 30]
BackOffice ['application/vnd.google-apps.spreadsheet', 1]
Ci ['application/vnd.google-apps.document', 1]
Human Resources ['application/vnd.google-apps.document', 1]
Human Resources ['application/vnd.google-apps.spreadsheet', 1]
IT-ОТДЕЛ ['application/vnd.google-apps.spreadsheet', 1]
Managment 2 ['application/vnd.google-apps.document', 1]
Managment 2 ['application/vnd.google-apps.shortcut', 1]
V-CLOUD24 ['application/vnd.google-apps.site', 1]
VG.COM ['application/vnd.google-apps.spreadsheet', 1]
Архив ['application/vnd.google-apps.document', 1]
Бухгалтер RUS ['application/vnd.google-apps.document', 11]
Бухгалтер RUS ['application/vnd.google-apps.spreadsheet', 2]
ВДНХ_Колесо обозрения ['application/vnd.google-apps.document', 1]
ВДНХ_Колесо обозрения ['application/vnd.google-apps.shortcut', 15]
Контент ['application/vnd.google-apps.document', 64]
Контент ['application/vnd.google-apps.spreadsheet', 7]
Лобня ['application/vnd.google-ap

In [28]:
mime_type = 'application/vnd.google-apps.document'
response = service.files().list(
        q=f"mimeType='{mime_type}'",
        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='files(id, name, mimeType)').execute()

In [24]:
len(fl)

30

In [7]:
ls1 = []
for file in response.get('files', []):
    ls1.append([file.get('id'), file.get('name'), file.get('mimeType')])

In [44]:
def downloadFile(file_name, file_id, mimeType, service):
    request = service.files().get_media(fileId=file_id)
    if "application/vnd.google-apps" in mimeType:
        if "document" in mimeType:
            request = service.files().export_media(fileId=file_id, mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')
            file_name = file_name + ".docx"
        else:
            request = service.files().export_media(fileId=file_id, mimeType='application/pdf')
            file_name = file_name + ".pdf"
    print("Downloading -- " + file_name)
    response = request.execute()
    with open(file_name, "wb") as wer:
        wer.write(response)

In [45]:
downloadFile(fl1[1], fl1[0], fl1[2], service)

In [42]:
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print('Download %d%%.' % int(status.progress() * 100))


HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1SKwTfRGC4Td_4K5ClSIlOLRsiE59vLIwSegzgovQOGw?alt=media returned "Only files with binary content can be downloaded. Use Export with Docs Editors files.". Details: "Only files with binary content can be downloaded. Use Export with Docs Editors files.">

In [41]:
fh = io.FileIO(fl1[1] + '.docx', mode='wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print('Download %d%%.' % int(status.progress() * 100))

HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1SKwTfRGC4Td_4K5ClSIlOLRsiE59vLIwSegzgovQOGw?alt=media returned "Only files with binary content can be downloaded. Use Export with Docs Editors files.". Details: "Only files with binary content can be downloaded. Use Export with Docs Editors files.">

In [33]:
if request:
    filename = 'your-file-name.docx'
    with open(filename, 'wb') as pdf_file:
        pdf_file.write(request)

TypeError: a bytes-like object is required, not 'HttpRequest'

In [22]:
request = service.files().export_media(fileId=fl1_id,
                                             mimeType=fl1_mime)

In [30]:
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

HttpError: <HttpError 400 when requesting https://www.googleapis.com/drive/v3/files/1SKwTfRGC4Td_4K5ClSIlOLRsiE59vLIwSegzgovQOGw/export?mimeType=application%2Fvnd.google-apps.document returned "The requested conversion is not supported.". Details: "The requested conversion is not supported.">

In [28]:
fil_id = ls_files[-1]
fil_id

'1tCX9WLNqv_Cp8HoBzcN67yL_g5mUOWcC'

In [29]:
file_path = get_full_path_to_file(service, drive_name_id, fil_id)
file_path

'GoogleDriveTransfer/BackOffice/Регламенты/СТП/Утвержденные/Утвержденные/Рабочие инструкции/РИ 05.13/Листок учёта.xlsx'

In [24]:
data = service.files().get_media(fileId=fil_id).execute()

In [11]:
blobs = bucket.list_blobs(prefix='GoogleDriveTransfer/BackOffice')

In [12]:
files = [blob.name for blob in blobs]

In [7]:
ls1 =  []
for driver in drivers:
    ls_files = get_list_of_files_in_drive(service, driver)
    ls1.append([driver[0], len(ls_files)])

In [8]:
ls2 = []
for driver in drivers:
    blobs = bucket.list_blobs(prefix=f'GoogleDriveTransfer/{driver[0]}')
    files = [blob.name for blob in blobs]
    ls2.append([driver[0], len(files)])

In [9]:
ls1

[['Ci', 7952],
 ['Managment 2', 21687],
 ['VG.COM', 16407],
 ['Архив', 34661],
 ['Винный парк', 7761],
 ['Контент_VG', 2845],
 ['Контент', 9500],
 ['Лобня', 5584],
 ['Объединенная пивоварня', 14814],
 ['ОСК Шереметьево', 1094]]

In [10]:
ls2

[['Ci', 2345],
 ['Managment 2', 14070],
 ['VG.COM', 16395],
 ['Архив', 1020],
 ['Винный парк', 1217],
 ['Контент_VG', 2531],
 ['Контент', 12029],
 ['Лобня', 5575],
 ['Объединенная пивоварня', 14772],
 ['ОСК Шереметьево', 718]]

In [8]:
blobs = bucket.list_blobs(prefix=f'GoogleDriveTransfer/{drivers[0][0]}')

In [9]:
files = [blob.name for blob in blobs]

In [10]:
drivers[0][0]

'BackOffice'

In [5]:
ls1

[['BackOffice', 1005],
 ['Grand_Engineering', 69],
 ['Human Resources', 1932],
 ['IT-ОТДЕЛ', 47],
 ['Managment 1', 7856],
 ['Titov Maksim', 93],
 ['V-CLOUD24', 27],
 ['Адвокатский кабинет', 857],
 ['Бухгалтер RUS', 8979],
 ['Бухгалтер UA', 1984],
 ['ВДНХ_Колесо обозрения', 809],
 ['ГипроМез', 2902],
 ['ДОУ 360 Выпуск', 164],
 ['ДОУ 360', 2829],
 ['Лио-Продукт', 575]]